<a href="https://colab.research.google.com/github/twelvesense/first-repository/blob/master/roi_trans_fair1m_mm_rr_le_lcg_2022_0917.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1: 기본 환경 설정 (Google Drive 연결 포함)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os, sys
try:
  Path_aiffelthon = '/content/1_aiffelthon'
  Path_mmrotate = '/content/1_mmrotate'
  Path_train = '/content/3_train'
  Path_infer = '/content/3_infer'
  Path_infer_wo_ann = '/content/3_infer_wo_ann'
  Path_roi_trans = '/content/2_roi_trans'
  os.symlink('/content/drive/MyDrive/Aiffelthon', Path_aiffelthon)  
  os.symlink('/content/drive/MyDrive/Aiffelthon/mmrotate', Path_mmrotate)  
  os.symlink('/content/drive/MyDrive/Aiffelthon/mmrotate/_train', Path_train)  
  os.symlink('/content/drive/MyDrive/Aiffelthon/mmrotate/_infer', Path_infer)  
  os.symlink('/content/drive/MyDrive/Aiffelthon/mmrotate/_infer_wo_ann', Path_infer_wo_ann)  
  os.symlink('/content/drive/MyDrive/Aiffelthon/mmrotate/configs/roi_trans', Path_roi_trans)  
except FileExistsError:
  Path_aiffelthon = '/content/1_aiffelthon'
  Path_mmrotate = '/content/1_mmrotate'
  Path_train = '/content/3_train'
  Path_infer = '/content/3_infer'
  Path_infer_wo_ann = '/content/3_infer_wo_ann'
  Path_roi_trans = '/content/2_roi_trans'

# Install MMCV and MMDetection using MIM.
%cd /content/drive/MyDrive/Aiffelthon
!pip3 install -U openmim
!mim install mmcv-full==1.6.0
!mim install mmdet
# Install MMRotate from the source.
# !git clone https://github.com/open-mmlab/mmrotate.git
%cd /content/drive/MyDrive/Aiffelthon/mmrotate
!pip install -e .

import mmcv
from mmcv.runner import load_checkpoint
from mmcv import collect_env
collect_env()

# Check MMRotate installation
import mmrotate
from mmrotate.models import build_detector
print(mmrotate.__version__)

# Check MMDetection installation
import mmdet
from mmdet.apis import inference_detector, show_result_pyplot
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# WandB installation
!pip install wandb
import wandb
print(wandb.__version__)

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Aiffelthon
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.7 MB/s 
     |████████████████████████████████| 235 kB 8.0 MB/s 
     |████████████████████████████████| 51 kB 8.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     |████████████████████████████████| 40.1 MB 11.6 MB/s 
     |████████████████████████████████| 190 kB 4.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     |████████████████████████████████| 1.4 MB 3.2 MB/s 
/content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Ai

# 2: WandB log-in

In [ ]:
# !wandb login  
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# 3: 폴더/파일명 설정 (개인이름 initial 기록 요망!!!)

In [ ]:
# 모델config파일 수정을 위한 configs폴더로의 이동 (ctrl + click)
/content/2_roi_trans

In [ ]:
# 1: 모델 CONFIG 파일 지정 (**특히, dataset 폴더 지정**)
CFG_FILE = 'roi_trans/roi_trans_r50_fpn_1x_fair1m_ms_rr_le90_lcg.py'
## CFG_FILE = 'roi_trans/roi_trans_r50_fpn_1x_fair1m_ms_rr_le90_yms.py'
## CFG_FILE = 'roi_trans/roi_trans_r50_fpn_1x_fair1m_ms_rr_le90_jsk.py'

# 2: 모델 TRAIN 결과 폴더 지정 (mmrotate 폴더 아래)
# TRAIN_DIRS = '_train/_train_lcg/fair1m_roi-trans_ms_rr_le90_10'
TRAIN_DIRS = '_train/_train_lcg/fair1m_roi-trans_ms_rr_le90_4-2a'
## TRAIN_DIRS = '_train/_train_jsk/fair1m_roi-trans_ms_rr_le90'
## TRAIN_DIRS = '_train/_train_yms/fair1m_roi-trans_ms_rr_le90'

# 3: 모델 INFERENCE (with ann.) 결과 폴더 지정 (mmroate 폴더 아래)
# INFER_DIRS = '_infer/_infer_lcg/fair1m_roi-trans_mm_rr_le90_VAL'
# INFER_DIRS = '_infer/_infer_lcg/fair1m_roi-trans_mm_rr_le90_VAL_super10'
# INFER_DIRS = '_infer/_infer_lcg/fair1m_roi-trans_mm_rr_le90_VAL_super10_a'
INFER_DIRS = '_infer/_infer_lcg'
# INFER_DIRS = '_infer/_infer_lcg/fair1m_roi-trans_mm_rr_le90_TEST'
## INFER_DIRS = '_infer/_infer_jsk/fair1m_roi-trans_mm_rr_le90'
## INFER_DIRS = '_infer/_infer_yms/fair1m_roi-trans_mm_rr_le90'

os.environ["CFG_FILE"] = CFG_FILE
os.environ["TRAIN_DIRS"] = TRAIN_DIRS
os.environ["INFER_DIRS"] = INFER_DIRS

# 4: to TRAIN

## 4-1: python script 파일 이용 (둘중 하나만 선택)

In [ ]:
os.environ["CFG_MODE"] = 'TRAIN_MODE'
!python ./tools/train.py \
./configs/$CFG_FILE \
--work-dir=$TRAIN_DIRS

2022-09-20 01:13:39,907 - mmrotate - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.14 (default, Sep  8 2022, 00:06:44) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla P100-PCIE-16GB
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.1, V11.1.105
GCC: x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.12.1+cu113
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencod

### **WandB 종료**

In [ ]:
wandb.finish()

## 4-2: jupyter notebook 이용 (둘중 하나만 선택)

In [ ]:
import argparse
import copy
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
import torch.distributed as dist
from mmcv import Config, DictAction
from mmcv.runner import get_dist_info, init_dist
from mmcv.utils import get_git_hash
from mmdet import __version__
from mmdet.apis import init_random_seed, set_random_seed

from mmrotate.apis import train_detector
from mmrotate.datasets import build_dataset
from mmrotate.models import build_detector
from mmrotate.utils import collect_env, get_root_logger, setup_multi_processes

def parse_args():
    parser = argparse.ArgumentParser(description='Train a detector')
    parser.add_argument('config', help='train config file path')
    parser.add_argument('--work-dir', help='the dir to save logs and models')
    parser.add_argument(
        '--resume-from', help='the checkpoint file to resume from')
    parser.add_argument(
        '--auto-resume',
        action='store_true',
        help='resume from the latest checkpoint automatically')
    parser.add_argument(
        '--no-validate',
        action='store_true',
        help='whether not to evaluate the checkpoint during training')
    group_gpus = parser.add_mutually_exclusive_group()
    group_gpus.add_argument(
        '--gpus',
        type=int,
        help='number of gpus to use '
        '(only applicable to non-distributed training)')
    group_gpus.add_argument(
        '--gpu-ids',
        type=int,
        nargs='+',
        help='ids of gpus to use '
        '(only applicable to non-distributed training)')
    parser.add_argument('--seed', type=int, default=None, help='random seed')
    parser.add_argument(
        '--diff-seed',
        action='store_true',
        help='Whether or not set different seeds for different ranks')
    parser.add_argument(
        '--deterministic',
        action='store_true',
        help='whether to set deterministic options for CUDNN backend.')
    parser.add_argument(
        '--cfg-options',
        nargs='+',
        action=DictAction,
        help='override some settings in the used config, the key-value pair '
        'in xxx=yyy format will be merged into config file. If the value to '
        'be overwritten is a list, it should be like key="[a,b]" or key=a,b '
        'It also allows nested list/tuple values, e.g. key="[(a,b),(c,d)]" '
        'Note that the quotation marks are necessary and that no white space '
        'is allowed.')
    parser.add_argument(
        '--launcher',
        choices=['none', 'pytorch', 'slurm', 'mpi'],
        default='none',
        help='job launcher')
    parser.add_argument('--local_rank', type=int, default=0)
    args = parser.parse_args(['./configs/'+CFG_FILE, '--work-dir='+TRAIN_DIRS])
    if 'LOCAL_RANK' not in os.environ:
        os.environ['LOCAL_RANK'] = str(args.local_rank)

    return args
    
%cd /content/drive/MyDrive/Aiffelthon/mmrotate

args = parse_args()

cfg = Config.fromfile(args.config)
if args.cfg_options is not None:
    cfg.merge_from_dict(args.cfg_options)

# set multi-process settings
setup_multi_processes(cfg)

# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

# work_dir is determined in this priority: CLI > segment in file > filename
if args.work_dir is not None:
    # update configs according to CLI args if args.work_dir is not None
    cfg.work_dir = args.work_dir
elif cfg.get('work_dir', None) is None:
    # use config filename as default work_dir if cfg.work_dir is None
    cfg.work_dir = osp.join('./_train',
                            osp.splitext(osp.basename(args.config))[0])
if args.resume_from is not None:
    cfg.resume_from = args.resume_from
cfg.auto_resume = args.auto_resume
if args.gpu_ids is not None:
    cfg.gpu_ids = args.gpu_ids
else:
    cfg.gpu_ids = range(1) if args.gpus is None else range(args.gpus)

# init distributed env first, since logger depends on the dist info.
if args.launcher == 'none':
    distributed = False
    if len(cfg.gpu_ids) > 1:
        warnings.warn(
            f'We treat {cfg.gpu_ids} as gpu-ids, and reset to '
            f'{cfg.gpu_ids[0:1]} as gpu-ids to avoid potential error in '
            'non-distribute training time.')
        cfg.gpu_ids = cfg.gpu_ids[0:1]
else:
    distributed = True
    init_dist(args.launcher, **cfg.dist_params)
    # re-set gpu_ids with distributed training mode
    _, world_size = get_dist_info()
    cfg.gpu_ids = range(world_size)

# create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# dump config
cfg.dump(osp.join(cfg.work_dir, osp.basename(args.config)))
# init the logger before other steps
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
log_file = osp.join(cfg.work_dir, f'{timestamp}.log')
logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)

# init the meta dict to record some important information such as
# environment info and seed, which will be logged
meta = dict()
# log env info
env_info_dict = collect_env()
env_info = '\n'.join([(f'{k}: {v}') for k, v in env_info_dict.items()])
dash_line = '-' * 60 + '\n'
logger.info('Environment info:\n' + dash_line + env_info + '\n' +
            dash_line)
meta['env_info'] = env_info
meta['config'] = cfg.pretty_text
# log some basic info
logger.info(f'Distributed training: {distributed}')
logger.info(f'Config:\n{cfg.pretty_text}')

# set random seeds
seed = init_random_seed(args.seed)
seed = seed + dist.get_rank() if args.diff_seed else seed
logger.info(f'Set random seed to {seed}, '
            f'deterministic: {args.deterministic}')
set_random_seed(seed, deterministic=args.deterministic)
cfg.seed = seed
meta['seed'] = seed
meta['exp_name'] = osp.basename(args.config)

model = build_detector(
    cfg.model,
    train_cfg=cfg.get('train_cfg'),
    test_cfg=cfg.get('test_cfg'))
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
if len(cfg.workflow) == 2:
    val_dataset = copy.deepcopy(cfg.data.val)
    val_dataset.pipeline = cfg.data.train.pipeline
    datasets.append(build_dataset(val_dataset))
if cfg.checkpoint_config is not None:
    # save mmdet version, config file content and class names in
    # checkpoints as meta data
    cfg.checkpoint_config.meta = dict(
        mmdet_version=__version__ + get_git_hash()[:7],
        CLASSES=datasets[0].CLASSES)
# add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

/content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Aiffelthon/mmrotate


2022-09-20 01:38:04,314 - mmrotate - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.14 (default, Sep  8 2022, 00:06:44) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla P100-PCIE-16GB
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.1, V11.1.105
GCC: x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.12.1+cu113
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencod

### **학습개시**

In [ ]:
os.environ["CFG_MODE"] = 'TRAIN_MODE'
%debug
train_detector(
    model,
    datasets,
    cfg,
    distributed=distributed,
    validate=(not args.no_validate),
    timestamp=timestamp,
    meta=meta)

> /usr/lib/python3.7/threading.py(296)wait()
    294         try:    # restore state no matter what (e.g., KeyboardInterrupt)
    295             if timeout is None:
--> 296                 waiter.acquire()
    297                 gotit = True
    298             else:

ipdb> w
  <ipython-input-51-dd98d769c837>(10)<module>()
      6     cfg,
      7     distributed=distributed,
      8     validate=(not args.no_validate),
      9     timestamp=timestamp,
---> 10     meta=meta)

  /content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Aiffelthon/mmrotate/mmrotate/apis/train.py(141)train_detector()
    137     if cfg.resume_from:
    138         runner.resume(cfg.resume_from)
    139     elif cfg.load_from:
    140         runner.load_checkpoint(cfg.load_from)
--> 141     runner.run(data_loaders, cfg.workflow)

  /usr/local/lib/python3.7/dist-packages/mmcv/runner/epoch_based_runner.py(136)run()
    134                     if mode == 'train' and self.epoch >= self._max_e

2022-09-20 01:55:42,149 - mmrotate - INFO - Start running, host: root@cd9c938b2207, work_dir: /content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Aiffelthon/mmrotate/_train/_train_lcg/fair1m_roi-trans_ms_rr_le90_4-2a
2022-09-20 01:55:42,152 - mmrotate - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
 ------------

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2/2, 4.6 task/s, elapsed: 0s, ETA:     0s

2022-09-20 01:55:57,588 - mmrotate - INFO - 
+----------+-----+------+--------+-------+
| class    | gts | dets | recall | ap    |
+----------+-----+------+--------+-------+
| ship     | 1   | 9    | 0.000  | 0.000 |
| vehicle  | 20  | 214  | 0.000  | 0.000 |
| airplane | 0   | 22   | 0.000  | 0.000 |
+----------+-----+------+--------+-------+
| mAP      |     |      |        | 0.000 |
+----------+-----+------+--------+-------+
2022-09-20 01:55:57,643 - mmrotate - INFO - Exp name: roi_trans_r50_fpn_1x_fair1m_ms_rr_le90_lcg.py
2022-09-20 01:55:57,646 - mmrotate - INFO - Epoch(val) [1][2]	mAP: 0.0000


### **WandB 종료**

In [ ]:
wandb.finish()

# 5: to INFER

## 5-1: python script 파일 이용

### (1) with annotation files by folder [**tools/test.py**]

In [ ]:
os.environ["CFG_MODE"] = 'INFER_MODE'
!python ./tools/test.py       \
./configs/$CFG_FILE           \
./_train/fair1m_roi-trans_ms_rr_le90_jsk2/epoch_12.pth  \
--out $INFER_DIRS/output.pkl  \
--show                        \
--work-dir $INFER_DIRS        \
--show-dir $INFER_DIRS        \
--show-score-thr 0.0001       \
--tmpdir $INFER_DIRS          \
--eval metric

# !python ./tools/test.py       \
# ./configs/$CFG_FILE           \
# ./_train/_train_lcg/fair1m_roi-trans_ms_rr_le90/1epoch_11.pth  \
# --out $INFER_DIRS/output.pkl  \
# --show                        \
# --work-dir $INFER_DIRS        \
# --show-dir $INFER_DIRS        \
# --show-score-thr 0.0001       \
# --tmpdir $INFER_DIRS          \
# --eval metric

### (2) without annotation files by folder [**demo/image_demo_folder.py**]

In [ ]:
os.environ["CFG_MODE"] = 'INFER_MODE'
!python ./demo/image_demo_folder.py    \
./data/fair1m2.0/images_val/           \
configs/$CFG_FILE                      \
./_train/fair1m_roi-trans_ms_rr_le90_jsk2/epoch_12.pth  \
--out-file=./demo/                     \
--score-thr=0.0001                     \
--folder-input=./data/fair1m2.0/images_test_10_lcg \
--folder-output=./_infer_wo_ann/_infer_lcg/test_10

### (3) without annotation file by single image file [**demo/image_demo.py**]

In [ ]:
os.environ["CFG_MODE"] = 'INFER_MODE'
!python ./demo/image_demo.py             \
./data/fair1m2.0/images_val/1012.tif     \
configs/$CFG_FILE                        \
./_train/fair1m_roi-trans_ms_rr_le90_jsk2/epoch_12.pth  \
--out-file=./demo/fair1m_1012.tif        \
--score-thr=0.0001                  

# TOOL: image with bboxes

In [ ]:
%cd /content/drive/MyDrive/Aiffelthon/mmrotate
PATH_IMAGE = './data/fair1m2.0/images_val_without_airplane_600'
PATH_ANNO = './data/fair1m2.0/val_super_without_airplane_600'
PATH_OUTPUT = './_infer/_infer_lcg/fair1m_roi-trans_mm_rr_le90_VAL_super_without_airplane_600_0.0001_metric'

/content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Aiffelthon/mmrotate


In [ ]:
from glob import glob
import os        
list_images2 = []
list_annos2 = []
list_images = glob(PATH_IMAGE + '/*.tif')
list_images = sorted(list_images, reverse=False)
list_images = sorted(list_images, key=len)
list_annos = glob(PATH_ANNO + '/*.txt')
list_annos = sorted(list_annos, reverse=False)
list_annos = sorted(list_annos, key=len)
for i in range(len(list_images)):
    list_images2.append(list_images[i][len(PATH_IMAGE)+1:])
if len(list_images) != len(list_annos):
    print('ERROR: The numbers of the two must be same.')

In [ ]:
from PIL import Image
from PIL import ImageDraw

for j in range(len(list_images2)):
# for j in range(10):
  img = Image.open(list_images[j])
  draw = ImageDraw.Draw(img)
  f = open(list_annos[j], 'r')
  lines = f.readlines()

  for i in range(len(lines)):
      position = lines[i].find(' ')
      x1 = int(float(lines[i][:position]))
      lines[i] = lines[i][position+1:]
      position = lines[i].find(' ')
      y1 = int(float(lines[i][:position]))
      lines[i] = lines[i][position+1:]
      position = lines[i].find(' ')
      x2 = int(float(lines[i][:position]))
      lines[i] = lines[i][position+1:]
      position = lines[i].find(' ')
      y2 = int(float(lines[i][:position]))
      lines[i] = lines[i][position+1:]
      position = lines[i].find(' ')
      x3 = int(float(lines[i][:position]))
      lines[i] = lines[i][position+1:]
      position = lines[i].find(' ')
      y3 = int(float(lines[i][:position]))
      lines[i] = lines[i][position+1:]
      position = lines[i].find(' ')
      x4 = int(float(lines[i][:position]))
      lines[i] = lines[i][position+1:]
      position = lines[i].find(' ')
      y4 = int(float(lines[i][:position]))
      lines[i] = lines[i][position+1:]
      # print(x1, y1, x2, y2, x3, y3, x4, y4)
      draw.line([(x3, y3), (x4, y4), (x1, y1), (x2, y2), (x3, y3)], fill="yellow", width=1)

  f.close()
  img.save(PATH_OUTPUT + '/' + list_images2[j][0:list_images2[j].find('.')] + '_bbox.tif', 'TIFF')